# Cummulative number of occurrence per publishing country

This is a request that we got from NatureServe to help them with their metrics (https://bipdashboard.natureserve.org/bip/SelectCountry.html).
They needed to the number of records on GBIF per publishing country over the years.

In hive, in the analytics folder:

```
SELECT snapshot, publisher_country, sum(count) FROM occ_publishercountry_complete
GROUP BY snapshot, publisher_country
```
Then download table

In [1]:
import pandas as pd
import csv

In [2]:
hive_query = pd.read_table("analyitics_per_country_occ_count.csv", sep=",")

In [3]:
hive_query["month"] = pd.to_datetime(hive_query.snapshot).dt.month
hive_query["year"] = pd.to_datetime(hive_query.snapshot).dt.year

In [4]:
publisher_country = "publisher_country"

In [5]:
reshaped_table = pd.DataFrame(index=sorted(hive_query[publisher_country].fillna("NULL").unique().tolist()))

for year in hive_query.year.unique().tolist():
    data_for_that_year = hive_query[hive_query.year == year]
    max_month = data_for_that_year.month.max()
    data_for_that_month = data_for_that_year[data_for_that_year.month == max_month]
    data_for_that_month[publisher_country] = data_for_that_month[publisher_country].fillna("NULL")
    
    for country in data_for_that_month[publisher_country].tolist():
        reshaped_table.at[country, year] = data_for_that_month.loc[data_for_that_month[publisher_country] == country, "_c2"].values[0]

/Users/gjq527/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
reshaped_table.to_excel("cumulative_count_GBIF_publishing_country_oct2019.xlsx")